In [ ]:
#-*-coding:utf-8-*-
import util
import tensorflow as tf
MNIST_CLASSIFIER_FROZEN_GRAPH = './classify_mnist_graph_def.pb'
INPUT_TENSOR = 'inputs:0'
OUTPUT_TENSOR = 'logits:0'
def EuclideanDistances(A, B):

    BT = B.transpose()
    # vecProd = A * BT
    vecProd = np.dot(A, BT)
    # print(vecProd)
    SqA = A ** 2
    # print(SqA)
    sumSqA = np.matrix(np.sum(SqA, axis=1))
    sumSqAEx = np.tile(sumSqA.transpose(), (1, vecProd.shape[1]))
    # print(sumSqAEx)

    SqB = B ** 2
    sumSqB = np.sum(SqB, axis=1)
    sumSqBEx = np.tile(sumSqB, (vecProd.shape[0], 1))
    SqED = sumSqBEx + sumSqAEx - 2 * vecProd
    SqED[SqED < 0] = 0.0
    ED = np.sqrt(SqED)
    return np.divide(ED.sum(),ED.shape[0]*ED.shape[1])
def cal_distance_image_real(images,labels):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)
def cal_distance_image_fake(images):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    labels = tf.Session().run(tf.argmax(y_logits, 1))
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)

    

In [ ]:
import os
if not os.path.isdir('saved_models_{}'.format('bgan')):
    os.mkdir('saved_models_{}'.format('bgan'))
f = open('saved_models_{}/log_collapse1.txt'.format('bgan'), mode='w')
import torch.utils.data as Data

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import keras.backend as K

import matplotlib.pyplot as plt
import sys
import tensorflow as tf
import numpy as np
import os
class BGAN():
    """Reference: https://wiseodd.github.io/techblog/2017/03/07/boundary-seeking-gan/"""
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generated imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss=self.boundary_loss, optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def boundary_loss(self, y_true, y_pred):
        """
        Boundary seeking loss.
        Reference: https://wiseodd.github.io/techblog/2017/03/07/boundary-seeking-gan/
        """
        return 0.5 * K.mean((K.log(y_pred) - K.log(1 - y_pred))**2)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (X_test,y_test) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5
        # X_test = X_test / 127.5 - 1.
        X_test = np.expand_dims(X_test, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            # idx = np.random.randint(0, X_train.shape[0], batch_size)
            for index in range(nb_batches):
                global_step += 1
                # progress_bar.update(index)

                # get a batch of real images
                image_batch = X_train[index * batch_size:(index + 1) * batch_size]

                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # Generate a batch of new images
                gen_imgs = self.generator.predict(noise)

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch(image_batch, valid)
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                g_loss = self.combined.train_on_batch(noise, valid)

                # Plot the progress
                print("epoch:%d step:%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch,global_step, d_loss[0], 100 * d_loss[1], g_loss))
                sampleSize = 5000
                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    s = self.sample_images(global_step, X_test,y_test, sampleSize)
    def sample_images(self, epoch,x_test,y_test,sample_num):
        r, c = sample_num//10, 10
        noise = np.random.normal(0, 1, (r * c, 100))
#         sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        labels = np.squeeze(y_test[:sample_num])
        labels = np.squeeze(labels)
        dis_real = cal_distance_image_real(x_test[:sample_num], labels)
        dis_fake = cal_distance_image_fake(gen_imgs)
        dis_cha = dis_real - dis_fake
        print('##############')
        # print(dis_real)
        # print(dis_fake)
        print(dis_cha)
        print('##########')
        f.writelines('\n')
        f.writelines('epoch:' + str(epoch))
        f.writelines('\n')
        f.writelines('紧度')
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in dis_cha)
        f.writelines('\n')
        


if __name__ == '__main__':
    bgan = BGAN()
    bgan.train(epochs=30, batch_size=64, sample_interval=200)

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:1 [D loss: 0.449949, acc.: 53.91%] [G loss: 0.185739]
epoch:0 step:2 [D loss: 0.454315, acc.: 62.50%] [G loss: 0.112001]
epoch:0 step:3 [D loss: 0.350803, acc.: 78.91%] [G loss: 0.168612]
epoch:0 step:4 [D loss: 0.225489, acc.: 99.22%] [G loss: 0.509961]
epoch:0 step:5 [D loss: 0.178659, acc.: 99.22%] [G loss: 0.791281]
epoch:0 step:6 [D loss: 0.166051, acc.: 99.22%] [G loss: 0.984321]
epoch:0 step:7 [D loss: 0.139902, acc.: 99.22%] [G loss: 1.235713]
epoch:0 step:8 [D loss: 0.118457, acc.: 100.00%] [G loss: 1.460355]
epoch:0 step:9 [D loss: 0.117186, acc.: 100.00%] [G loss: 1.669372]
epoch:0 step:10 [D loss: 0.100809, acc.: 100.00%] [G loss: 1.825359]
epoch:0 step:11 [D loss: 0.084996, acc.: 100.00%] [G loss: 1.965951]
epoch:0 step:12 [D loss: 0.088377, acc.: 100.00%] [G loss: 2.101921]
epoch:0 step:13 [D loss: 0.087739, acc.: 100.00%] [G loss: 2.227346]
epoch:0 step:14 [D loss: 0.070623, acc.: 100.00%] [G loss: 2.574449]
epoch:0 step:15 [D loss: 0.065674, acc.: 100.00%] 

epoch:0 step:120 [D loss: 0.122556, acc.: 96.88%] [G loss: 6.466031]
epoch:0 step:121 [D loss: 0.054285, acc.: 99.22%] [G loss: 8.374132]
epoch:0 step:122 [D loss: 0.198350, acc.: 93.75%] [G loss: 5.125208]
epoch:0 step:123 [D loss: 0.084123, acc.: 98.44%] [G loss: 7.550138]
epoch:0 step:124 [D loss: 0.168070, acc.: 92.97%] [G loss: 7.225974]
epoch:0 step:125 [D loss: 0.058280, acc.: 100.00%] [G loss: 8.242271]
epoch:0 step:126 [D loss: 0.131117, acc.: 96.09%] [G loss: 8.772146]
epoch:0 step:127 [D loss: 0.065521, acc.: 99.22%] [G loss: 7.111512]
epoch:0 step:128 [D loss: 0.110426, acc.: 98.44%] [G loss: 7.548186]
epoch:0 step:129 [D loss: 0.079653, acc.: 98.44%] [G loss: 8.449320]
epoch:0 step:130 [D loss: 0.098329, acc.: 98.44%] [G loss: 8.352005]
epoch:0 step:131 [D loss: 0.118552, acc.: 98.44%] [G loss: 7.852937]
epoch:0 step:132 [D loss: 0.093813, acc.: 99.22%] [G loss: 8.743954]
epoch:0 step:133 [D loss: 0.321893, acc.: 88.28%] [G loss: 6.325898]
epoch:0 step:134 [D loss: 0.05718

...

epoch:24 step:22511 [D loss: 0.588687, acc.: 64.06%] [G loss: 0.774070]
epoch:24 step:22512 [D loss: 0.518395, acc.: 71.88%] [G loss: 0.742264]
epoch:24 step:22513 [D loss: 0.460375, acc.: 78.12%] [G loss: 0.755084]
epoch:24 step:22514 [D loss: 0.614522, acc.: 66.41%] [G loss: 0.660127]
epoch:24 step:22515 [D loss: 0.461684, acc.: 73.44%] [G loss: 0.736179]
epoch:24 step:22516 [D loss: 0.546098, acc.: 69.53%] [G loss: 0.740076]
epoch:24 step:22517 [D loss: 0.508503, acc.: 76.56%] [G loss: 0.701313]
epoch:24 step:22518 [D loss: 0.492132, acc.: 75.78%] [G loss: 0.685781]
epoch:24 step:22519 [D loss: 0.605145, acc.: 67.97%] [G loss: 0.600034]
epoch:24 step:22520 [D loss: 0.589109, acc.: 66.41%] [G loss: 0.587726]
epoch:24 step:22521 [D loss: 0.495015, acc.: 76.56%] [G loss: 0.688526]
epoch:24 step:22522 [D loss: 0.518202, acc.: 71.88%] [G loss: 0.633949]
epoch:24 step:22523 [D loss: 0.576704, acc.: 64.06%] [G loss: 0.561755]
epoch:24 step:22524 [D loss: 0.499678, acc.: 69.53%] [G loss: 0.

epoch:24 step:22624 [D loss: 0.516266, acc.: 75.78%] [G loss: 0.734579]
epoch:24 step:22625 [D loss: 0.578338, acc.: 66.41%] [G loss: 0.797301]
epoch:24 step:22626 [D loss: 0.518181, acc.: 72.66%] [G loss: 0.557114]
epoch:24 step:22627 [D loss: 0.550946, acc.: 73.44%] [G loss: 0.745426]
epoch:24 step:22628 [D loss: 0.519370, acc.: 72.66%] [G loss: 0.762894]
epoch:24 step:22629 [D loss: 0.531481, acc.: 73.44%] [G loss: 0.734882]
epoch:24 step:22630 [D loss: 0.588063, acc.: 64.06%] [G loss: 0.611781]
epoch:24 step:22631 [D loss: 0.559154, acc.: 68.75%] [G loss: 0.730136]
epoch:24 step:22632 [D loss: 0.460459, acc.: 79.69%] [G loss: 0.685505]
epoch:24 step:22633 [D loss: 0.544273, acc.: 69.53%] [G loss: 0.811034]
epoch:24 step:22634 [D loss: 0.444830, acc.: 79.69%] [G loss: 0.821915]
epoch:24 step:22635 [D loss: 0.600715, acc.: 64.84%] [G loss: 0.635890]
epoch:24 step:22636 [D loss: 0.580181, acc.: 63.28%] [G loss: 0.631517]
epoch:24 step:22637 [D loss: 0.482856, acc.: 73.44%] [G loss: 0.

epoch:24 step:22739 [D loss: 0.703958, acc.: 63.28%] [G loss: 0.910503]
epoch:24 step:22740 [D loss: 0.510121, acc.: 75.00%] [G loss: 0.635426]
epoch:24 step:22741 [D loss: 0.615809, acc.: 64.84%] [G loss: 0.630282]
epoch:24 step:22742 [D loss: 0.522893, acc.: 70.31%] [G loss: 0.603757]
epoch:24 step:22743 [D loss: 0.539361, acc.: 68.75%] [G loss: 0.658565]
epoch:24 step:22744 [D loss: 0.561223, acc.: 68.75%] [G loss: 0.682789]
epoch:24 step:22745 [D loss: 0.577900, acc.: 65.62%] [G loss: 0.539952]
epoch:24 step:22746 [D loss: 0.517178, acc.: 68.75%] [G loss: 0.725246]
epoch:24 step:22747 [D loss: 0.529661, acc.: 72.66%] [G loss: 0.619667]
epoch:24 step:22748 [D loss: 0.618547, acc.: 59.38%] [G loss: 0.596844]
epoch:24 step:22749 [D loss: 0.524789, acc.: 69.53%] [G loss: 0.597037]
epoch:24 step:22750 [D loss: 0.499754, acc.: 75.78%] [G loss: 0.768408]
epoch:24 step:22751 [D loss: 0.591584, acc.: 67.97%] [G loss: 0.506517]
epoch:24 step:22752 [D loss: 0.559858, acc.: 71.09%] [G loss: 0.

epoch:24 step:22851 [D loss: 0.560905, acc.: 69.53%] [G loss: 0.862592]
epoch:24 step:22852 [D loss: 0.505518, acc.: 70.31%] [G loss: 0.845927]
epoch:24 step:22853 [D loss: 0.557389, acc.: 67.97%] [G loss: 0.681997]
epoch:24 step:22854 [D loss: 0.514237, acc.: 77.34%] [G loss: 0.649610]
epoch:24 step:22855 [D loss: 0.628207, acc.: 67.97%] [G loss: 0.620267]
epoch:24 step:22856 [D loss: 0.545685, acc.: 66.41%] [G loss: 0.668700]
epoch:24 step:22857 [D loss: 0.545960, acc.: 73.44%] [G loss: 0.775224]
epoch:24 step:22858 [D loss: 0.550266, acc.: 71.09%] [G loss: 0.702996]
epoch:24 step:22859 [D loss: 0.515888, acc.: 74.22%] [G loss: 0.669640]
epoch:24 step:22860 [D loss: 0.559950, acc.: 69.53%] [G loss: 0.692115]
epoch:24 step:22861 [D loss: 0.553020, acc.: 68.75%] [G loss: 0.602998]
epoch:24 step:22862 [D loss: 0.460597, acc.: 78.91%] [G loss: 0.704170]
epoch:24 step:22863 [D loss: 0.544869, acc.: 71.88%] [G loss: 0.743864]
epoch:24 step:22864 [D loss: 0.631186, acc.: 64.84%] [G loss: 0.

epoch:24 step:22967 [D loss: 0.532278, acc.: 73.44%] [G loss: 0.641618]
epoch:24 step:22968 [D loss: 0.540857, acc.: 71.09%] [G loss: 0.662477]
epoch:24 step:22969 [D loss: 0.496177, acc.: 79.69%] [G loss: 0.676562]
epoch:24 step:22970 [D loss: 0.587482, acc.: 67.97%] [G loss: 0.678010]
epoch:24 step:22971 [D loss: 0.512811, acc.: 71.88%] [G loss: 0.581255]
epoch:24 step:22972 [D loss: 0.506059, acc.: 73.44%] [G loss: 0.881631]
epoch:24 step:22973 [D loss: 0.498084, acc.: 73.44%] [G loss: 0.863373]
epoch:24 step:22974 [D loss: 0.533020, acc.: 71.88%] [G loss: 0.594559]
epoch:24 step:22975 [D loss: 0.546593, acc.: 66.41%] [G loss: 0.619145]
epoch:24 step:22976 [D loss: 0.483711, acc.: 72.66%] [G loss: 0.740094]
epoch:24 step:22977 [D loss: 0.506999, acc.: 74.22%] [G loss: 0.555116]
epoch:24 step:22978 [D loss: 0.556526, acc.: 71.09%] [G loss: 0.629723]
epoch:24 step:22979 [D loss: 0.577973, acc.: 67.19%] [G loss: 0.594001]
epoch:24 step:22980 [D loss: 0.569500, acc.: 69.53%] [G loss: 0.